In [1]:
from joblib import load
import sys
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from scipy import signal

# Model Load

In [2]:
rfModel = load('models/random_forest.joblib')
mlpModel = load('models/neural_network.joblib')
svcModel = load('models/svc_classifier.joblib')

# Data Load

In [3]:
walkDF = []
standDF = []
runDF = []

for i in range(4):
    walkDF.append(pd.read_csv("data/testdata/walk{}.csv".format(i+1)))
    standDF.append(pd.read_csv("data/testdata/stand{}.csv".format(i+1)))
    runDF.append(pd.read_csv("data/testdata/run{}.csv".format(i+1)))

# Process Data
## Butterworth

In [4]:
# using low pass filter 
b, a = signal.butter(3, 0.1, btype='lowpass', analog=False)

walk_np=list(walkDF)
standing_np=list(standDF)
running_np=list(runDF)

butter_worth_walk = []
for i in range(4):
    butter_worth_walkX = signal.filtfilt(b, a, walk_np[i]["gFx"])
    butter_worth_walkY = signal.filtfilt(b, a, walk_np[i]["gFy"])
    butter_worth_walkZ = signal.filtfilt(b, a, walk_np[i]["gFz"])

    butter_worth_walk.append((pd.DataFrame(list(zip(walk_np[i]['time'],butter_worth_walkX, butter_worth_walkY, butter_worth_walkZ,walk_np[i]['acc'])),
                                            columns=["Time","AccX", "AccY", "AccZ","Acc-old"])))

butter_worth_standing = []
for i in range(4):
    butter_worth_standingX = signal.filtfilt(b, a, standing_np[i]["gFx"])
    butter_worth_standingY = signal.filtfilt(b, a, standing_np[i]["gFy"])
    butter_worth_standingZ = signal.filtfilt(b, a, standing_np[i]["gFz"])
    
    
    butter_worth_standing.append((pd.DataFrame(list(zip(standing_np[i]['time'],butter_worth_standingX, butter_worth_standingY, butter_worth_standingZ,standing_np[i]['acc'])),
                                            columns=["Time","AccX", "AccY", "AccZ","Acc-old"])))

butter_worth_running = []
for i in range(4):
    butter_worth_runningX = signal.filtfilt(b, a, running_np[i]["gFx"])
    butter_worth_runningY = signal.filtfilt(b, a, running_np[i]["gFy"])
    butter_worth_runningZ = signal.filtfilt(b, a, running_np[i]["gFz"])
    #butter_worth_walk_L =signal.filtfilt(b, a,walk_np[i]["acc"])
    
    
    butter_worth_running.append((pd.DataFrame(list(zip(running_np[i]['time'],butter_worth_runningX, butter_worth_runningY, butter_worth_runningZ,running_np[i]['acc'])),
                                            columns=["Time","AccX", "AccY", "AccZ","Acc-old"])))
       

for i in range(4):
    butter_worth_walk[i]["Acc-trns"] = np.sqrt(butter_worth_walk[i]["AccX"]**2 * 
                                                     butter_worth_walk[i]["AccY"]**2 *
                                                     butter_worth_walk[i]["AccZ"]**2 )

    butter_worth_standing[i]["Acc-trns"] = np.sqrt(butter_worth_standing[i]["AccX"]**2 * 
                                                     butter_worth_standing[i]["AccY"]**2 *
                                                     butter_worth_standing[i]["AccZ"]**2 )
    
    butter_worth_running[i]["Acc-trns"] = np.sqrt(butter_worth_running[i]["AccX"]**2 * 
                                                     butter_worth_running[i]["AccY"]**2 *
                                                     butter_worth_running[i]["AccZ"]**2 ) 
    


# Update current data with cleaned data
newWalk = []
newStand = []
newRun = []

for i in range(4):
    newWalk.append(butter_worth_walk[i])
    newStand.append(butter_worth_standing[i])
    newRun.append(butter_worth_running[i])

walkDF = newWalk
standDF = newStand
runDF = newRun

## Statistics

### Finding mean, min and max for Standing

In [5]:
df=pd.DataFrame(standDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df2=pd.DataFrame(standDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df3=pd.DataFrame(standDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df4=pd.DataFrame(standDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T

dfmin1=pd.DataFrame(standDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin2=pd.DataFrame(standDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin3=pd.DataFrame(standDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin4=pd.DataFrame(standDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T

dfmax1=pd.DataFrame(standDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax2=pd.DataFrame(standDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax3=pd.DataFrame(standDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax4=pd.DataFrame(standDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T

# append
Meanstanding=df.append(df2)
Meanstanding=Meanstanding.append(df3)
Meanstanding=Meanstanding.append(df4)

Minstanding=dfmin1.append(dfmin2)
Minstanding=Minstanding.append(dfmin3)
Minstanding=Minstanding.append(dfmin4)

Maxstanding=dfmax1.append(dfmax2)
Maxstanding=Maxstanding.append(dfmax3)
Maxstanding=Maxstanding.append(dfmax4)

Meanstanding.columns=['mean1AccX','mean1AccY','mean1AccZ','mean1Acc-old','mean1Acc-turns']
Minstanding.columns=['min1AccX','min1AccY','min1AccZ','min1Acc-old','min1Acc-turns']
Maxstanding.columns=['max1AccX','max1AccY','max1AccZ','max1Acc-old','max1Acc-turns']

Meanstanding['min1AccX']=Minstanding['min1AccX']
Meanstanding['min1AccY']=Minstanding['min1AccY']
Meanstanding['min1AccZ']=Minstanding['min1AccZ']
Meanstanding['min1Acc-old']=Minstanding['min1Acc-old']
Meanstanding['min1Acc-turns']=Minstanding['min1Acc-turns']
Meanstanding['max1AccX']=Maxstanding['max1AccX']
Meanstanding['max1AccY']=Maxstanding['max1AccY']
Meanstanding['max1AccZ']=Maxstanding['max1AccZ']
Meanstanding['max1Acc-old']=Maxstanding['max1Acc-old']
Meanstanding['max1Acc-turns']=Maxstanding['max1Acc-turns']

Meanstanding=Meanstanding.reset_index()
Meanstanding=Meanstanding.drop(columns=['index'])
standDF = Meanstanding

### Finding mean, min and max for walking

In [6]:
df=pd.DataFrame(walkDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df2=pd.DataFrame(walkDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df3=pd.DataFrame(walkDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df4=pd.DataFrame(walkDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T

dfmin1=pd.DataFrame(walkDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin2=pd.DataFrame(walkDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin3=pd.DataFrame(walkDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin4=pd.DataFrame(walkDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T

dfmax1=pd.DataFrame(walkDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax2=pd.DataFrame(walkDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax3=pd.DataFrame(walkDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax4=pd.DataFrame(walkDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T

# append
Meanwalking=df.append(df2)
Meanwalking=Meanwalking.append(df3)
Meanwalking=Meanwalking.append(df4)

Minwalking=dfmin1.append(dfmin2)
Minwalking=Minwalking.append(dfmin3)
Minwalking=Minwalking.append(dfmin4)

Maxwalking=dfmax1.append(dfmax2)
Maxwalking=Maxwalking.append(dfmax3)
Maxwalking=Maxwalking.append(dfmax4)

Meanwalking.columns=['mean1AccX','mean1AccY','mean1AccZ','mean1Acc-old','mean1Acc-turns']
Minwalking.columns=['min1AccX','min1AccY','min1AccZ','min1Acc-old','min1Acc-turns']
Maxwalking.columns=['max1AccX','max1AccY','max1AccZ','max1Acc-old','max1Acc-turns']

Meanwalking['min1AccX']=Minwalking['min1AccX']
Meanwalking['min1AccY']=Minwalking['min1AccY']
Meanwalking['min1AccZ']=Minwalking['min1AccZ']
Meanwalking['min1Acc-old']=Minwalking['min1Acc-old']
Meanwalking['min1Acc-turns']=Minwalking['min1Acc-turns']
Meanwalking['max1AccX']=Maxwalking['max1AccX']
Meanwalking['max1AccY']=Maxwalking['max1AccY']
Meanwalking['max1AccZ']=Maxwalking['max1AccZ']
Meanwalking['max1Acc-old']=Maxwalking['max1Acc-old']
Meanwalking['max1Acc-turns']=Maxwalking['max1Acc-turns']

Meanwalking=Meanwalking.reset_index()
Meanwalking=Meanwalking.drop(columns=['index'])
walkDF = Meanwalking

### Finding mean, min and max for runing

In [7]:
df=pd.DataFrame(runDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df2=pd.DataFrame(runDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df3=pd.DataFrame(runDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T
df4=pd.DataFrame(runDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].mean()).T

dfmin1=pd.DataFrame(runDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin2=pd.DataFrame(runDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin3=pd.DataFrame(runDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T
dfmin4=pd.DataFrame(runDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].min()).T

dfmax1=pd.DataFrame(runDF[0][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax2=pd.DataFrame(runDF[1][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax3=pd.DataFrame(runDF[2][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T
dfmax4=pd.DataFrame(runDF[3][['AccX','AccY','AccZ','Acc-old','Acc-trns']].max()).T

# append
Meanruning=df.append(df2)
Meanruning=Meanruning.append(df3)
Meanruning=Meanruning.append(df4)

Minruning=dfmin1.append(dfmin2)
Minruning=Minruning.append(dfmin3)
Minruning=Minruning.append(dfmin4)

Maxruning=dfmax1.append(dfmax2)
Maxruning=Maxruning.append(dfmax3)
Maxruning=Maxruning.append(dfmax4)

Meanruning.columns=['mean1AccX','mean1AccY','mean1AccZ','mean1Acc-old','mean1Acc-turns']
Minruning.columns=['min1AccX','min1AccY','min1AccZ','min1Acc-old','min1Acc-turns']
Maxruning.columns=['max1AccX','max1AccY','max1AccZ','max1Acc-old','max1Acc-turns']

Meanruning['min1AccX']=Minruning['min1AccX']
Meanruning['min1AccY']=Minruning['min1AccY']
Meanruning['min1AccZ']=Minruning['min1AccZ']
Meanruning['min1Acc-old']=Minruning['min1Acc-old']
Meanruning['min1Acc-turns']=Minruning['min1Acc-turns']
Meanruning['max1AccX']=Maxruning['max1AccX']
Meanruning['max1AccY']=Maxruning['max1AccY']
Meanruning['max1AccZ']=Maxruning['max1AccZ']
Meanruning['max1Acc-old']=Maxruning['max1Acc-old']
Meanruning['max1Acc-turns']=Maxruning['max1Acc-turns']

Meanruning=Meanruning.reset_index()
Meanruning=Meanruning.drop(columns=['index'])
runDF = Meanruning

# Predictions

### Standing

In [8]:
y_rf = rfModel.predict(standDF)
y_mlp = mlpModel.predict(standDF)
y_svc = svcModel.predict(standDF)

standResults = pd.DataFrame()
standResults['Random Forest'] = y_rf
standResults['MLP'] = y_mlp
standResults['SVC'] = y_svc
print("Classifier-wise Predictions\n")
print(standResults)

y_rf_prob = pd.DataFrame(rfModel.predict_proba(standDF))
y_rf_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nRandom Forest Probability\n")
print(y_rf_prob)

y_mlp_prob = pd.DataFrame(mlpModel.predict_proba(standDF))
y_mlp_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nMLP Probability\n")
print(y_mlp_prob)

y_svc_prob = pd.DataFrame(svcModel.predict_proba(standDF))
y_svc_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nSVC Probability\n")
print(y_svc_prob)

Classifier-wise Predictions

   Random Forest  MLP  SVC
0              0    0    0
1              0    0    0
2              0    0    0
3              0    0    0


Random Forest Probability

   Standing   Walking  Running
0  1.000000  0.000000      0.0
1  0.925627  0.074373      0.0
2  1.000000  0.000000      0.0
3  1.000000  0.000000      0.0


MLP Probability

   Standing       Walking       Running
0  0.999997  2.577931e-06  8.510257e-27
1  1.000000  6.080674e-08  4.570902e-28
2  1.000000  3.102523e-07  1.695445e-28
3  0.999997  3.225492e-06  5.465279e-26


SVC Probability

   Standing   Walking   Running
0  0.912098  0.069849  0.018053
1  0.916553  0.062405  0.021042
2  0.923110  0.059816  0.017074
3  0.893725  0.086560  0.019715


### Walking

In [9]:
# Walking
y_rf = rfModel.predict(walkDF)
y_mlp = mlpModel.predict(walkDF)
y_svc = svcModel.predict(walkDF)

walkResults = pd.DataFrame()
walkResults['Random Forest'] = y_rf
walkResults['MLP'] = y_mlp
walkResults['SVC'] = y_svc
print("Classifier-wise Predictions\n")
print(walkResults)

y_rf_prob = pd.DataFrame(rfModel.predict_proba(walkDF))
y_rf_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nRandom Forest Probability\n")
print(y_rf_prob)

y_mlp_prob = pd.DataFrame(mlpModel.predict_proba(walkDF))
y_mlp_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nMLP Probability\n")
print(y_mlp_prob)

y_svc_prob = pd.DataFrame(svcModel.predict_proba(walkDF))
y_svc_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nSVC Probability\n")
print(y_svc_prob)

Classifier-wise Predictions

   Random Forest  MLP  SVC
0              1    1    1
1              1    1    1
2              1    1    1
3              1    1    1


Random Forest Probability

   Standing   Walking   Running
0  0.000000  1.000000  0.000000
1  0.169355  0.535842  0.294803
2  0.000000  1.000000  0.000000
3  0.042652  0.957348  0.000000


MLP Probability

       Standing   Walking       Running
0  9.597772e-07  0.999999  3.489074e-13
1  9.001181e-11  0.999908  9.222985e-05
2  7.080348e-06  0.999993  4.768719e-15
3  7.323623e-06  0.999993  3.849002e-14


SVC Probability

   Standing   Walking   Running
0  0.077870  0.831764  0.090366
1  0.080544  0.691732  0.227724
2  0.096058  0.844253  0.059689
3  0.085726  0.831295  0.082979


### Running

In [10]:
y_rf = rfModel.predict(runDF)
y_mlp = mlpModel.predict(runDF)
y_svc = svcModel.predict(runDF)

runResults = pd.DataFrame()
runResults['Random Forest'] = y_rf
runResults['MLP'] = y_mlp
runResults['SVC'] = y_svc
print("Classifier-wise Predictions\n")
print(runResults)

y_rf_prob = pd.DataFrame(rfModel.predict_proba(runDF))
y_rf_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nRandom Forest Probability\n")
print(y_rf_prob)

y_mlp_prob = pd.DataFrame(mlpModel.predict_proba(runDF))
y_mlp_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nMLP Probability\n")
print(y_mlp_prob)

y_svc_prob = pd.DataFrame(svcModel.predict_proba(runDF))
y_svc_prob.columns = ['Standing', 'Walking', 'Running']
print("\n\nSVC Probability\n")
print(y_svc_prob)

Classifier-wise Predictions

   Random Forest  MLP  SVC
0              2    2    2
1              2    2    2
2              2    2    2
3              2    2    2


Random Forest Probability

   Standing   Walking   Running
0    0.0000  0.000000  1.000000
1    0.0319  0.034409  0.933692
2    0.0000  0.000000  1.000000
3    0.0000  0.000000  1.000000


MLP Probability

       Standing       Walking  Running
0  1.552181e-49  3.021296e-12      1.0
1  2.147260e-29  4.013973e-08      1.0
2  1.459241e-59  2.607529e-13      1.0
3  1.355531e-56  1.575476e-10      1.0


SVC Probability

   Standing   Walking   Running
0  0.010511  0.059156  0.930333
1  0.010674  0.090366  0.898960
2  0.006909  0.020574  0.972517
3  0.006313  0.031554  0.962133
